# Task_Midterm_Report. 
   
## Procedures
<ol>
<li> Collect SARS-CoV2 genes
<li> Do Multiple Sequence Alignment & Phylogeny Construction.
<li> Calculate the mutation rate estimate of SARS-CoV2
<li> Measure the mutation rate estimate for each gene
</ol>

## Question
<ol>
<li> Using multiple sequence analysis and phylogenetic tree reconstruction, can you
estimate the propagation path of SARS-CoV-2 in the US? Justify your idea with
the data.
<li> Based on the number of new mutations & the collection time, can you estimate
the “mutation rate”? (how long does it take to make a new mutation?)
<li> Is there any specific gene having higher rate of mutation? Can you explain why?
</ol>

In [24]:
# The function to read sequences from a FASTA file
def read_fasta(tmp):
    r = dict()
    fi = open(tmp, 'r')
    for line in fi:
        if line.startswith('>'):
            h_g = line.strip().lstrip('>')
            r[h_g] = ''
        else:
            r[h_g] += line.strip().replace(' ', '')
    fi.close()
    return r

In [25]:
# 1-(1) : Read genomes

SARS_CoV2_genomes = '2019nCoV_genomes.2020_04_14.clean.fa'
genome_COVID = read_fasta(SARS_CoV2_genomes)
CoV2_rep_idx = '../data/GCF_009858895.2_ASM985889v3_genomic.gff'
ref_genome = '../data/GCF_009858895.2_ASM985889v3_genomic.fna'
genome_COVID_ref = read_fasta(ref_genome)

In [26]:
# 1-(2) : get index

def get_idx(t, p):
    fi = open(t, 'r')
    for line in fi.readlines():
        if "gene" in line and "Name={}".format(p) in line:
            idx_s = int(line.split()[3])-1
            idx_e = int(line.split()[4])-1
            idx = (idx_s, idx_e)
    return idx
N_idx = get_idx(CoV2_rep_idx, 'N')

In [27]:
# 1-(3) : Get the cDNA seq of N protein in SARS-CoV2

for seq in genome_COVID_ref.values():
    ref_N_cDNA = seq[N_idx[0]:N_idx[1]+1]
ref_Np_cDNA = {'N_prot_cDNA' : [ref_N_cDNA]}
def w_seq(fi, gc):
    f = open(fi, 'w')
    for g in list(gc.keys()):
        if len(gc[g]) == 1:
            f.write(">{}\n".format(g))
            f.write("{}\n".format(gc[g][0]))
        else:
            for i in range(len(gc[g])):
                f.write(">{} #{}\n".format(g, i+1))
                f.write("\n{}".format(gc[g][i]))
    f.close()
w_seq("ref_Np_cDNA.fa", ref_Np_cDNA)

In [28]:
# 1-(4) : Blast about SARS-CoV2 N gene and Make .fa file about N gene sequence data of each 2019nCoV_genomes.2020_04_14.clean.fa

'''
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ makeblastdb -dbtype nucl -in 2019nCoV_genomes.2020_04_14.clean.fa -out 2019nCoV_genomes.2020_04_14.clean
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ blastn -db 2019nCoV_genomes.2020_04_14.clean -query ref_Np_cDNA.fa -outfmt 6 -out 2019nCoV_genomes_N.2020_04_14_blastn.out -max_target_seqs 746
'''

cD_set = open("SARS_CoV2_N_set.fa", 'w')
gf = open('2019nCoV_genomes_N.2020_04_14_blastn.out', 'r')

val_list = []
lines = gf.readlines()

for line in lines:
    for val in list(genome_COVID.keys()):
        if line.split()[1] in val:
            val_list.append(line.split()[1])

val_set = set(val_list)

cD_dic = {val : [] for val in val_set}

for line in lines:
    cD_dic[line.split()[1]].append(line.split()[8])
    cD_dic[line.split()[1]].append(line.split()[9])
          
        
for val in cD_dic.keys():
    if len(cD_dic[val]) > 2:
        max_idx = max(cD_dic[val])
        min_idx = min(cD_dic[val])
        cD_dic[val].clear()
        cD_dic[val].append(min_idx)
        cD_dic[val].append(max_idx)
        
for val in cD_dic.keys():
    for key in genome_COVID.keys():
        if val in key:
            idx_1 = int(cD_dic[val][0])-1
            idx_2 = int(cD_dic[val][1])
            sen = ">%s\n%s\n" % (val, genome_COVID[key][idx_1:idx_2])
            cD_set.write(sen)


gf.close()
cD_set.close()

In [ ]:
# 1-(5) : Do multiple alignment and construct phylogenetic tree about N gene by using Mega X

In [1]:
# But, Phylogenetic tree from N gene has low accuracy because N gene is just only one gene of some genes in SARS-CoV2
# Thus, I'll select the nine genes(S, ORF1ab, ORF8, E, M, ORF3a, ORF6, ORF7a) and reconstruct phylogenetic tree from them

In [29]:
# 1-(6) :  reference sequence about S & ORF1ab & ORF8 & E & M & ORF3a & ORF6 & ORF7a

S_idx = get_idx(CoV2_rep_idx, 'S')
ORF8_idx = get_idx(CoV2_rep_idx, 'YP_009724396.1')
E_idx = get_idx(CoV2_rep_idx, 'E')
M_idx = get_idx(CoV2_rep_idx, 'M')
ORF1ab_idx = get_idx(CoV2_rep_idx, 'orf1ab')
ORF3a_idx = get_idx(CoV2_rep_idx, 'ORF3a')
M_idx = get_idx(CoV2_rep_idx, 'M')
ORF6_idx = get_idx(CoV2_rep_idx, 'ORF6')
ORF7a_idx = get_idx(CoV2_rep_idx, 'ORF7a')


for seq in genome_COVID_ref.values():
    ref_S_cDNA = seq[S_idx[0]:S_idx[1]+1]
ref_Sp_cDNA = {'S_prot_cDNA' : [ref_S_cDNA]}

for seq in genome_COVID_ref.values():
    ref_ORF8_cDNA = seq[ORF8_idx[0]:ORF8_idx[1]+1]
ref_ORF8p_cDNA = {'ORF8_prot_cDNA' : [ref_ORF8_cDNA]}

for seq in genome_COVID_ref.values():
    ref_E_cDNA = seq[E_idx[0]:E_idx[1]+1]
ref_Ep_cDNA = {'E_prot_cDNA' : [ref_E_cDNA]}

for seq in genome_COVID_ref.values():
    ref_ORF1ab_cDNA = seq[ORF1ab_idx[0]:ORF1ab_idx[1]+1]
ref_ORF1abp_cDNA = {'ORF1ab_prot_cDNA' : [ref_ORF1ab_cDNA]}

for seq in genome_COVID_ref.values():
    ref_ORF3a_cDNA = seq[ORF3a_idx[0]:ORF3a_idx[1]+1]
ref_ORF3ap_cDNA = {'ORF3a_prot_cDNA' : [ref_ORF3a_cDNA]}

for seq in genome_COVID_ref.values():
    ref_M_cDNA = seq[M_idx[0]:M_idx[1]+1]
ref_Mp_cDNA = {'M_prot_cDNA' : [ref_M_cDNA]}

for seq in genome_COVID_ref.values():
    ref_ORF6_cDNA = seq[ORF6_idx[0]:ORF6_idx[1]+1]
ref_ORF6p_cDNA = {'ORF6_prot_cDNA' : [ref_ORF6_cDNA]}

for seq in genome_COVID_ref.values():
    ref_ORF7a_cDNA = seq[ORF7a_idx[0]:ORF7a_idx[1]+1]
ref_ORF7ap_cDNA = {'ORF7a_prot_cDNA' : [ref_ORF7a_cDNA]}



w_seq("ref_Sp_cDNA.fa", ref_Sp_cDNA)
w_seq("ref_ORF8p_cDNA.fa", ref_ORF8p_cDNA)
w_seq("ref_Ep_cDNA.fa", ref_Ep_cDNA)
w_seq("ref_ORF1abp_cDNA.fa", ref_ORF1abp_cDNA)
w_seq("ref_ORF3ap_cDNA.fa", ref_ORF3ap_cDNA)
w_seq("ref_Mp_cDNA.fa", ref_Mp_cDNA)
w_seq("ref_ORF6p_cDNA.fa", ref_ORF6p_cDNA)
w_seq("ref_ORF7ap_cDNA.fa", ref_ORF7ap_cDNA)

In [30]:
# 1-(7) : Blast about nine genes except for N gene

'''
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ blastn -db 2019nCoV_genomes.2020_04_14.clean -query ref_Sp_cDNA.fa -outfmt 6 -out 2019nCoV_genomes_S.2020_04_14_blastn.out -max_target_seqs 746
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ blastn -db 2019nCoV_genomes.2020_04_14.clean -query ref_ORF8p_cDNA.fa -outfmt 6 -out 2019nCoV_genomes_ORF8.2020_04_14_blastn.out -max_target_seqs 746
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ blastn -db 2019nCoV_genomes.2020_04_14.clean -query ref_Ep_cDNA.fa -outfmt 6 -out 2019nCoV_genomes_E.2020_04_14_blastn.out -max_target_seqs 746
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ blastn -db 2019nCoV_genomes.2020_04_14.clean -query ref_ORF1abp_cDNA.fa -outfmt 6 -out 2019nCoV_genomes_ORF1ab.2020_04_14_blastn.out -max_target_seqs 746
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ blastn -db 2019nCoV_genomes.2020_04_14.clean -query ref_ORF3ap_cDNA.fa -outfmt 6 -out 2019nCoV_genomes_ORF3a.2020_04_14_blastn.out -max_target_seqs 746
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ blastn -db 2019nCoV_genomes.2020_04_14.clean -query ref_Mp_cDNA.fa -outfmt 6 -out 2019nCoV_genomes_M.2020_04_14_blastn.out -max_target_seqs 746
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ blastn -db 2019nCoV_genomes.2020_04_14.clean -query ref_ORF6p_cDNA.fa -outfmt 6 -out 2019nCoV_genomes_ORF6.2020_04_14_blastn.out -max_target_seqs 746
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ blastn -db 2019nCoV_genomes.2020_04_14.clean -query ref_ORF7ap_cDNA.fa -outfmt 6 -out 2019nCoV_genomes_ORF7a.2020_04_14_blastn.out -max_target_seqs 746
'''

# 1-(8) : Make .fa file about sequence data(from nine genes) of each 2019nCoV_genomes.2020_04_14.clean.fa
# > S gene

cD_set_S = open("SARS_CoV2_S_set.fa", 'w')
gf_S = open('2019nCoV_genomes_S.2020_04_14_blastn.out', 'r')

val_list = []
lines = gf_S.readlines()

for line in lines:
    for val in list(genome_COVID.keys()):
        if line.split()[1] in val:
            val_list.append(line.split()[1])

val_set = set(val_list)

cD_dic = {val : [] for val in val_set}

for line in lines:
    cD_dic[line.split()[1]].append(line.split()[8])
    cD_dic[line.split()[1]].append(line.split()[9])
          
        
for val in cD_dic.keys():
    if len(cD_dic[val]) > 2:
        max_idx = max(cD_dic[val])
        min_idx = min(cD_dic[val])
        cD_dic[val].clear()
        cD_dic[val].append(min_idx)
        cD_dic[val].append(max_idx)
        
for val in cD_dic.keys():
    for key in genome_COVID.keys():
        if val in key:
            idx_1 = int(cD_dic[val][0])-1
            idx_2 = int(cD_dic[val][1])
            sen = ">%s\n%s\n" % (val, genome_COVID[key][idx_1:idx_2])
            cD_set_S.write(sen)

gf_S.close()
cD_set_S.close()   

In [31]:
# > E gene & ORF8 & ORF1ab

cD_set_E = open("SARS_CoV2_E_set.fa", 'w')
gf_E = open('2019nCoV_genomes_E.2020_04_14_blastn.out', 'r')

val_list = []
lines = gf_E.readlines()

for line in lines:
    for val in list(genome_COVID.keys()):
        if line.split()[1] in val:
            val_list.append(line.split()[1])

val_set = set(val_list)

cD_dic = {val : [] for val in val_set}

for line in lines:
    cD_dic[line.split()[1]].append(line.split()[8])
    cD_dic[line.split()[1]].append(line.split()[9])
          
        
for val in cD_dic.keys():
    if len(cD_dic[val]) > 2:
        max_idx = max(cD_dic[val])
        min_idx = min(cD_dic[val])
        cD_dic[val].clear()
        cD_dic[val].append(min_idx)
        cD_dic[val].append(max_idx)
        
for val in cD_dic.keys():
    for key in genome_COVID.keys():
        if val in key:
            idx_1 = int(cD_dic[val][0])-1
            idx_2 = int(cD_dic[val][1])
            sen = ">%s\n%s\n" % (val, genome_COVID[key][idx_1:idx_2])
            cD_set_E.write(sen)

gf_E.close()
cD_set_E.close()

cD_set_ORF8 = open("SARS_CoV2_ORF8_set.fa", 'w')
gf_ORF8 = open('2019nCoV_genomes_ORF8.2020_04_14_blastn.out', 'r')

val_list = []
lines = gf_ORF8.readlines()

for line in lines:
    for val in list(genome_COVID.keys()):
        if line.split()[1] in val:
            val_list.append(line.split()[1])

val_set = set(val_list)

cD_dic = {val : [] for val in val_set}

for line in lines:
    cD_dic[line.split()[1]].append(line.split()[8])
    cD_dic[line.split()[1]].append(line.split()[9])
          
        
for val in cD_dic.keys():
    if len(cD_dic[val]) > 2:
        max_idx = max(cD_dic[val])
        min_idx = min(cD_dic[val])
        cD_dic[val].clear()
        cD_dic[val].append(min_idx)
        cD_dic[val].append(max_idx)
        
for val in cD_dic.keys():
    for key in genome_COVID.keys():
        if val in key:
            idx_1 = int(cD_dic[val][0])-1
            idx_2 = int(cD_dic[val][1])
            sen = ">%s\n%s\n" % (val, genome_COVID[key][idx_1:idx_2])
            cD_set_ORF8.write(sen)

gf_ORF8.close()
cD_set_ORF8.close()

cD_set_ORF1ab = open("SARS_CoV2_ORF1ab_d_set.fa", 'w')
gf_ORF1ab = open('2019nCoV_genomes_ORF1ab.2020_04_14_blastn.out', 'r')

val_list = []
lines = gf_ORF1ab.readlines()

for line in lines:
    for val in list(genome_COVID.keys()):
        if line.split()[1] in val:
            val_list.append(line.split()[1])

val_set = set(val_list)

cD_dic = {val : [] for val in val_set}

for line in lines:
    cD_dic[line.split()[1]].append(line.split()[8])
    cD_dic[line.split()[1]].append(line.split()[9])
          
        
for val in cD_dic.keys():
    if len(cD_dic[val]) > 2:
        max_idx = max(cD_dic[val])
        min_idx = min(cD_dic[val])
        cD_dic[val].clear()
        cD_dic[val].append(min_idx)
        cD_dic[val].append(max_idx)
        
for val in cD_dic.keys():
    for key in genome_COVID.keys():
        if val in key:
            idx_1 = int(cD_dic[val][0])-1
            idx_2 = int(cD_dic[val][1])
            sen = ">%s\n%s\n" % (val, genome_COVID[val][idx_1:idx_2])
            cD_set_ORF1ab.write(sen)         
            
        
        

gf_ORF1ab.close()
cD_set_ORF1ab.close()

In [32]:
# > ORF3a & M gene

cD_set_ORF3a = open("SARS_CoV2_ORF3a_set.fa", 'w')
gf_ORF3a = open('2019nCoV_genomes_ORF3a.2020_04_14_blastn.out', 'r')

val_list = []
lines = gf_ORF3a.readlines()

for line in lines:
    for val in list(genome_COVID.keys()):
        if line.split()[1] in val:
            val_list.append(line.split()[1])

val_set = set(val_list)

cD_dic = {val : [] for val in val_set}

for line in lines:
    cD_dic[line.split()[1]].append(line.split()[8])
    cD_dic[line.split()[1]].append(line.split()[9])
          
        
for val in cD_dic.keys():
    if len(cD_dic[val]) > 2:
        max_idx = max(cD_dic[val])
        min_idx = min(cD_dic[val])
        cD_dic[val].clear()
        cD_dic[val].append(min_idx)
        cD_dic[val].append(max_idx)
        
for val in cD_dic.keys():
    for key in genome_COVID.keys():
        if val in key:
            idx_1 = int(cD_dic[val][0])-1
            idx_2 = int(cD_dic[val][1])
            sen = ">%s\n%s\n" % (val, genome_COVID[key][idx_1:idx_2])
            cD_set_ORF3a.write(sen)

gf_ORF3a.close()
cD_set_ORF3a.close()

cD_set_M = open("SARS_CoV2_M_set.fa", 'w')
gf_M = open('2019nCoV_genomes_M.2020_04_14_blastn.out', 'r')

val_list = []
lines = gf_M.readlines()

for line in lines:
    for val in list(genome_COVID.keys()):
        if line.split()[1] in val:
            val_list.append(line.split()[1])

val_set = set(val_list)

cD_dic = {val : [] for val in val_set}

for line in lines:
    cD_dic[line.split()[1]].append(line.split()[8])
    cD_dic[line.split()[1]].append(line.split()[9])
          
        
for val in cD_dic.keys():
    if len(cD_dic[val]) > 2:
        max_idx = max(cD_dic[val])
        min_idx = min(cD_dic[val])
        cD_dic[val].clear()
        cD_dic[val].append(min_idx)
        cD_dic[val].append(max_idx)
        
for val in cD_dic.keys():
    for key in genome_COVID.keys():
        if val in key:
            idx_1 = int(cD_dic[val][0])-1
            idx_2 = int(cD_dic[val][1])
            sen = ">%s\n%s\n" % (val, genome_COVID[key][idx_1:idx_2])
            cD_set_M.write(sen)

gf_M.close()
cD_set_M.close()

In [33]:
# > ORF6 & ORF7a

cD_set_ORF6 = open("SARS_CoV2_ORF6_set.fa", 'w')
gf_ORF6 = open('2019nCoV_genomes_ORF6.2020_04_14_blastn.out', 'r')

val_list = []
lines = gf_ORF6.readlines()

for line in lines:
    for val in list(genome_COVID.keys()):
        if line.split()[1] in val:
            val_list.append(line.split()[1])

val_set = set(val_list)

cD_dic = {val : [] for val in val_set}

for line in lines:
    cD_dic[line.split()[1]].append(line.split()[8])
    cD_dic[line.split()[1]].append(line.split()[9])
          
        
for val in cD_dic.keys():
    if len(cD_dic[val]) > 2:
        max_idx = max(cD_dic[val])
        min_idx = min(cD_dic[val])
        cD_dic[val].clear()
        cD_dic[val].append(min_idx)
        cD_dic[val].append(max_idx)
        
for val in cD_dic.keys():
    for key in genome_COVID.keys():
        if val in key:
            idx_1 = int(cD_dic[val][0])-1
            idx_2 = int(cD_dic[val][1])
            sen = ">%s\n%s\n" % (val, genome_COVID[key][idx_1:idx_2])
            cD_set_ORF6.write(sen)

gf_ORF6.close()
cD_set_ORF6.close()

cD_set_ORF7a = open("SARS_CoV2_ORF7a_set.fa", 'w')
gf_ORF7a = open('2019nCoV_genomes_ORF7a.2020_04_14_blastn.out', 'r')

val_list = []
lines = gf_ORF7a.readlines()

for line in lines:
    for val in list(genome_COVID.keys()):
        if line.split()[1] in val:
            val_list.append(line.split()[1])

val_set = set(val_list)

cD_dic = {val : [] for val in val_set}

for line in lines:
    cD_dic[line.split()[1]].append(line.split()[8])
    cD_dic[line.split()[1]].append(line.split()[9])
          
        
for val in cD_dic.keys():
    if len(cD_dic[val]) > 2:
        max_idx = max(cD_dic[val])
        min_idx = min(cD_dic[val])
        cD_dic[val].clear()
        cD_dic[val].append(min_idx)
        cD_dic[val].append(max_idx)
        
for val in cD_dic.keys():
    for key in genome_COVID.keys():
        if val in key:
            idx_1 = int(cD_dic[val][0])-1
            idx_2 = int(cD_dic[val][1])
            sen = ">%s\n%s\n" % (val, genome_COVID[key][idx_1:idx_2])
            cD_set_ORF7a.write(sen)

gf_ORF7a.close()
cD_set_ORF7a.close()

In [34]:
# > Insert missing ORF1ab sequence data -> (But, I don't know why some data is missing)

cD_ORF1ab = "SARS_CoV2_ORF1ab_d_set.fa"
ORF1ab = read_fasta(cD_ORF1ab)

ORF1ab_dic = {}

for gORF1ab in list(ORF1ab.keys()):
    for key in genome_COVID.keys():
        if gORF1ab in key:
            ORF1ab_dic[key] = ORF1ab[gORF1ab]
            
for n_key in ORF1ab_dic.keys():
    if n_key == 'MT292579|Spain|2020-03-09':
        ORF1ab_dic[n_key] = genome_COVID['MT292579|Spain|2020-03-09'][211:21501]
    elif n_key == 'MT326115|USA|2020-03-20':
        ORF1ab_dic[n_key] = genome_COVID['MT326115|USA|2020-03-20'][0:21249]
    elif n_key == 'MT322399|USA_VA|2020-03-00':
        ORF1ab_dic[n_key] = genome_COVID['MT322399|USA_VA|2020-03-00'][226:21516]
    elif n_key == 'MT198651|Spain_Valencia|2020-03-04':
        ORF1ab_dic[n_key] = genome_COVID['MT198651|Spain_Valencia|2020-03-04'][211:21501]
    elif n_key == 'MT259272|USA_WA|2020-03-13':
        ORF1ab_dic[n_key] = genome_COVID['MT259272|USA_WA|2020-03-13'][0:21136]
    elif n_key == 'MT326183|USA|2020-03-17':
        ORF1ab_dic[n_key] = genome_COVID['MT326183|USA|2020-03-17'][119:21407]
    elif n_key == 'MT326038|USA|2020-03-24':
        ORF1ab_dic[n_key] = genome_COVID['MT326038|USA|2020-03-24'][0:21184]
    elif n_key == 'MT326108|USA|2020-03-22':
        ORF1ab_dic[n_key] = genome_COVID['MT326108|USA|2020-03-22'][0:21251]
    elif n_key == 'MT308695|USA_MI|2020-03-10':
        ORF1ab_dic[n_key] = genome_COVID['MT308695|USA_MI|2020-03-10'][216:21506]
    elif n_key == 'MT263407|USA_WA|2020-03-22':
        ORF1ab_dic[n_key] = genome_COVID['MT263407|USA_WA|2020-03-22'][0:21279]
    elif n_key == 'MT326142|USA|2020-03-19':
        ORF1ab_dic[n_key] = genome_COVID['MT326142|USA|2020-03-19'][0:21053]
    elif n_key == 'MT259242|USA_WA|2020-03-14':
        ORF1ab_dic[n_key] = genome_COVID['MT259242|USA_WA|2020-03-14'][0:20911]
    elif n_key == 'MT326131|USA|2020-03-20':
        ORF1ab_dic[n_key] = genome_COVID['MT326131|USA|2020-03-20'][0:21098]
    elif n_key == 'MT246458|USA_WA|2020-03-12':
        ORF1ab_dic[n_key] = genome_COVID['MT246458|USA_WA|2020-03-12'][139:21429]
    elif n_key == 'MT259238|USA_WA|2020-03-16':
        ORF1ab_dic[n_key] = genome_COVID['MT259238|USA_WA|2020-03-16'][0:20813]
    elif n_key == 'MT263397|USA_WA|2020-03-24':
        ORF1ab_dic[n_key] = genome_COVID['MT263397|USA_WA|2020-03-24'][0:21254]
    elif n_key == 'MT263426|USA_WA|2020-03-24':
        ORF1ab_dic[n_key] = genome_COVID['MT263426|USA_WA|2020-03-24'][154:21444]
    elif n_key == 'MT326146|USA|2020-03-21':
        ORF1ab_dic[n_key] = genome_COVID['MT326146|USA|2020-03-21'][0:21190]
    elif n_key == 'MT263453|USA_WA|2020-03-24':
        ORF1ab_dic[n_key] = genome_COVID['MT263453|USA_WA|2020-03-24'][139:21429]
    elif n_key == 'MT322400|USA_VA|2020-03-00':
        ORF1ab_dic[n_key] = genome_COVID['MT322400|USA_VA|2020-03-00'][249:21539]
    elif n_key == 'MT259284|USA_WA|2020-03-15':
        ORF1ab_dic[n_key] = genome_COVID['MT259284|USA_WA|2020-03-15'][0:21156]
    elif n_key == 'MT308692|USA_MI|2020-03-09':
        ORF1ab_dic[n_key] = genome_COVID['MT308692|USA_MI|2020-03-09'][218:21508]
    elif n_key == 'MT326102|USA|2020-03-22':
        ORF1ab_dic[n_key] = genome_COVID['MT326102|USA|2020-03-22'][19:21309]
    elif n_key == 'MT308696|USA_MI|2020-03-09':
        ORF1ab_dic[n_key] = genome_COVID['MT308696|USA_MI|2020-03-09'][216:21506]
    elif n_key == 'MT246483|USA_WA|2020-03-15':
        ORF1ab_dic[n_key] = genome_COVID['MT246483|USA_WA|2020-03-15'][196:21429]
    elif n_key == 'MT263389|USA_WA|2020-03-23':
        ORF1ab_dic[n_key] = genome_COVID['MT263389|USA_WA|2020-03-23'][0:21106]
    elif n_key == 'MT293203|USA_WA|2020-03-30':
        ORF1ab_dic[n_key] = genome_COVID['MT293203|USA_WA|2020-03-30'][0:21251]
    elif n_key == 'MT308700|USA_MI|2020-03-12':
        ORF1ab_dic[n_key] = genome_COVID['MT308700|USA_MI|2020-03-12'][216:21506]
    elif n_key == 'MT246463|USA_WA|2020-03-13':
        ORF1ab_dic[n_key] = genome_COVID['MT246463|USA_WA|2020-03-13'][139:21429]
    elif n_key == 'MT326145|USA|2020-03-21':
        ORF1ab_dic[n_key] = genome_COVID['MT326145|USA|2020-03-21'][0:21060]
    elif n_key == 'MT259283|USA_WA|2020-03-15':
        ORF1ab_dic[n_key] = genome_COVID['MT259283|USA_WA|2020-03-15'][0:20939]
    elif n_key == 'MT308693|USA_MI|2020-03-10':
        ORF1ab_dic[n_key] = genome_COVID['MT308693|USA_MI|2020-03-10'][216:21506]
    elif n_key == 'MT263461|USA_WA|2020-03-22':
        ORF1ab_dic[n_key] = genome_COVID['MT263461|USA_WA|2020-03-22'][235:21481]
    elif n_key == 'MT246465|USA_WA|2020-03-13':
        ORF1ab_dic[n_key] = genome_COVID['MT246465|USA_WA|2020-03-13'][241:21433]
    elif n_key == 'MT259270|USA_WA|2020-03-16':
        ORF1ab_dic[n_key] = genome_COVID['MT259270|USA_WA|2020-03-16'][0:21088]
    elif n_key == 'MT256918|Spain_Valencia|2020-03-06':
        ORF1ab_dic[n_key] = genome_COVID['MT256918|Spain_Valencia|2020-03-06'][211:21501]
    elif n_key == 'MT326101|USA|2020-03-22':
        ORF1ab_dic[n_key] = genome_COVID['MT326101|USA|2020-03-22'][250:21407]
    elif n_key == 'MT256917|Spain_Valencia|2020-03-02':
        ORF1ab_dic[n_key] = genome_COVID['MT256917|Spain_Valencia|2020-03-02'][211:21501]
    elif n_key == 'MT308699|USA_MI|2020-03-12':
        ORF1ab_dic[n_key] = genome_COVID['MT308699|USA_MI|2020-03-12'][210:21500]
    elif n_key == 'MT326186|USA|2020-03-17':
        ORF1ab_dic[n_key] = genome_COVID['MT326186|USA|2020-03-17'][0:21251]
    elif n_key == 'MT259255|USA_WA|2020-03-16':
        ORF1ab_dic[n_key] = genome_COVID['MT259255|USA_WA|2020-03-16'][0:21093]
    elif n_key == 'MT326032|USA|2020-03-23':
        ORF1ab_dic[n_key] = genome_COVID['MT326032|USA|2020-03-23'][274:21506]
    elif n_key == 'MT326126|USA|2020-03-20':
        ORF1ab_dic[n_key] = genome_COVID['MT326126|USA|2020-03-20'][300:21508]
    elif n_key == 'MT292580|Spain|2020-02-27':
        ORF1ab_dic[n_key] = genome_COVID['MT292580|Spain|2020-02-27'][211:21501]
    elif n_key == 'MT263427|USA_WA|2020-03-24':
        ORF1ab_dic[n_key] = genome_COVID['MT263427|USA_WA|2020-03-24'][320:21462]
    elif n_key == 'MT292582|Spain|2020-03-06':
        ORF1ab_dic[n_key] = genome_COVID['MT292582|Spain|2020-03-06'][211:21501]
    elif n_key == 'MT326176|USA|2020-03-19':
        ORF1ab_dic[n_key] = genome_COVID['MT326176|USA|2020-03-19'][0:20992]
    elif n_key == 'MT326057|USA|2020-03-21':
        ORF1ab_dic[n_key] = genome_COVID['MT326057|USA|2020-03-21'][0:21177]
    elif n_key == 'MT308694|USA_MI|2020-03-11':
        ORF1ab_dic[n_key] = genome_COVID['MT308694|USA_MI|2020-03-11'][216:21506]
    elif n_key == 'MT263460|USA_WA|2020-03-24':
        ORF1ab_dic[n_key] = genome_COVID['MT263460|USA_WA|2020-03-24'][124:21407]
    elif n_key == 'MT263393|USA_WA|2020-03-23':
        ORF1ab_dic[n_key] = genome_COVID['MT263393|USA_WA|2020-03-23'][0:21182]
    elif n_key == 'MT263387|USA_WA|2020-03-23':
        ORF1ab_dic[n_key] = genome_COVID['MT263387|USA_WA|2020-03-23'][203:21493]
    elif n_key == 'MT308697|USA_MI|2020-03-11':
        ORF1ab_dic[n_key] = genome_COVID['MT308697|USA_MI|2020-03-11'][218:21508]
    elif n_key == 'MT326080|USA|2020-03-24':
        ORF1ab_dic[n_key] = genome_COVID['MT326080|USA|2020-03-24'][43:21136]
    elif n_key == 'LR757997|China_Wuhan|2019-12-31':
        ORF1ab_dic[n_key] = genome_COVID['LR757997|China_Wuhan|2019-12-31'][229:21560]
    elif n_key == 'MT259259|USA_WA|2020-03-16':
        ORF1ab_dic[n_key] = genome_COVID['MT259259|USA_WA|2020-03-16'][0:21037]
    elif n_key == 'MT292581|Spain|2020-02-27':
        ORF1ab_dic[n_key] = genome_COVID['MT292581|Spain|2020-02-27'][211:21501]
    elif n_key == 'MT263466|USA_WA|2020-03-23':
        ORF1ab_dic[n_key] = genome_COVID['MT263466|USA_WA|2020-03-23'][99:21376]
    elif n_key == 'MT326179|USA|2020-03-19':
        ORF1ab_dic[n_key] = genome_COVID['MT326179|USA|2020-03-19'][99:21389]
    elif n_key == 'MT326182|USA|2020-03-13':
        ORF1ab_dic[n_key] = genome_COVID['MT326182|USA|2020-03-13'][51:21341]
    elif n_key == 'MT263401|USA_WA|2020-03-23':
        ORF1ab_dic[n_key] = genome_COVID['MT263401|USA_WA|2020-03-23'][0:21175]
    elif n_key == 'MT263409|USA_WA|2020-03-22':
        ORF1ab_dic[n_key] = genome_COVID['MT263409|USA_WA|2020-03-22'][0:21190]
    elif n_key == 'MT293180|USA_WA|2020-03-28':
        ORF1ab_dic[n_key] = genome_COVID['MT293180|USA_WA|2020-03-28'][43:21293]
    elif n_key == 'MT308698|USA_MI|2020-03-12':
        ORF1ab_dic[n_key] = genome_COVID['MT308698|USA_MI|2020-03-12'][216:21506]
    elif n_key == 'MT326073|USA|2020-03-21':
        ORF1ab_dic[n_key] = genome_COVID['MT326073|USA|2020-03-21'][0:21192]


cD_r_ORFab = open('SARS_CoV2_ORF1ab_set.fa', 'w')

for o in list(ORF1ab_dic.keys()):
    cD_r_ORFab.write(">{}\n".format(o))
    cD_r_ORFab.write("{}\n".format(ORF1ab_dic[o]))    
    
cD_r_ORFab.close()

In [35]:
# 1-(9) : Do multiple Alignment about nine genes and merge all aligned data from each gene into one .fa file

# (base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ muscle -in SARS_CoV2_ORF1ab_set.fa -out SARS_CoV2_ORF1ab_mul_al.fa
# (base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ muscle -in SARS_CoV2_ORF8_set.fa -out SARS_CoV2_ORF8_mul_al.fa
# (base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ muscle -in SARS_CoV2_N_set.fa -out SARS_CoV2_N_mul_al.fa
# (base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ muscle -in SARS_CoV2_E_set.fa -out SARS_CoV2_E_mul_al.fa
# (base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ muscle -in SARS_CoV2_S_set.fa -out SARS_CoV2_S_mul_al.fa
# (base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ muscle -in SARS_CoV2_ORF3a_set.fa -out SARS_CoV2_ORF3a_mul_al.fa
# (base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ muscle -in SARS_CoV2_M_set.fa -out SARS_CoV2_M_mul_al.fa
# (base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ muscle -in SARS_CoV2_ORF6_set.fa -out SARS_CoV2_ORF6_mul_al.fa
# (base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/05_Report$ muscle -in SARS_CoV2_ORF7a_set.fa -out SARS_CoV2_ORF7a_mul_al.fa

alig_N = "SARS_CoV2_N_mul_al.fa"
aN = read_fasta(alig_N)
alig_S = "SARS_CoV2_S_mul_al.fa"
aS = read_fasta(alig_S)
alig_E = "SARS_CoV2_E_mul_al.fa"
aE = read_fasta(alig_E)
alig_ORF8 = "SARS_CoV2_ORF8_mul_al.fa"
aORF8 = read_fasta(alig_ORF8)
alig_ORF1ab = "SARS_CoV2_ORF1ab_mul_al.fa"
aORF1ab = read_fasta(alig_ORF1ab)
alig_ORF3a = "SARS_CoV2_ORF3a_mul_al.fa"
aORF3a = read_fasta(alig_ORF3a)
alig_M = "SARS_CoV2_M_mul_al.fa"
aM = read_fasta(alig_M)
alig_ORF6 = "SARS_CoV2_ORF6_mul_al.fa"
aORF6 = read_fasta(alig_ORF6)
alig_ORF7a = "SARS_CoV2_ORF7a_mul_al.fa"
aORF7a = read_fasta(alig_ORF7a)

cD_treeg_dic = {}

for gORF1ab in list(aORF1ab.keys()):
    if gORF1ab == 'NC_045512|China|2019-12-00':
        cD_treeg_dic[gORF1ab] = aORF1ab[gORF1ab]

for gORF1ab in list(aORF1ab.keys()):
    if gORF1ab != 'NC_045512|China|2019-12-00':
        cD_treeg_dic[gORF1ab] = aORF1ab[gORF1ab]
    
for gS in list(aS.keys()):
    for val in cD_treeg_dic.keys():
        if gS == val:
            cD_treeg_dic[val] += aS[gS]
            
for gORF3a in list(aORF3a.keys()):
    for val in cD_treeg_dic.keys():
        if gORF3a == val:
            cD_treeg_dic[val] += aORF3a[gORF3a]            
            
for gE in list(aE.keys()):
    for val in cD_treeg_dic.keys():
        if gE == val:
            cD_treeg_dic[val] += aE[gE]
            
for gM in list(aM.keys()):
    for val in cD_treeg_dic.keys():
        if gM == val:
            cD_treeg_dic[val] += aM[gM]

for gORF6 in list(aORF6.keys()):
    for val in cD_treeg_dic.keys():
        if gORF6 == val:
            cD_treeg_dic[val] += aORF6[gORF6]
            
for gORF7a in list(aORF7a.keys()):
    for val in cD_treeg_dic.keys():
        if gORF7a == val:
            cD_treeg_dic[val] += aORF7a[gORF7a]

for gORF8 in list(aORF8.keys()):
    for val in cD_treeg_dic.keys():
        if gORF8 == val:
            cD_treeg_dic[val] += aORF8[gORF8]

for gN in list(aN.keys()):
    for val in cD_treeg_dic.keys():
        if gN == val:
            cD_treeg_dic[val] += aN[gN]

cD_set_treeg = open('SARS_CoV2_rep_mul_align_set.fa', 'w')

for g in list(cD_treeg_dic.keys()):
    cD_set_treeg.write(">{}\n".format(g))
    cD_set_treeg.write("{}\n".format(cD_treeg_dic[g]))
    
cD_set_treeg.close()

In [191]:
# 1-(10) : Construct phylogenetic tree about nine genes by using mega X
# From the phylogenetic tree, the most of viruses in USA is from China
# More detailed explanation about infection path from phylogenetic tree is written on my midterm homework word file

In [36]:
# 2-(1) : Calculate mutation rate
# Assume that the reference genome is NC_045512|China|2019-12-00 because all parts of sequence data are same with reference sequence about nine genes

rep_align = 'SARS_CoV2_rep_mul_align_set.fa'
rep = read_fasta(rep_align)

ref_f_coll = rep['NC_045512|China|2019-12-00']

count = {name : [0] for name in rep.keys()}
N_or_gap_count = {name : [0] for name in rep.keys()}

del count['NC_045512|China|2019-12-00']
del N_or_gap_count['NC_045512|China|2019-12-00']

for name in list(rep.keys()):
    if name != 'NC_045512|China|2019-12-00':
        for i in range(len(ref_f_coll)):
            if rep[name][i] == 'N' or ref_f_coll[i] == 'N' or rep[name][i] == '-' or ref_f_coll[i] == '-':
                N_or_gap_count[name][0] += 1
            elif rep[name][i] != ref_f_coll[i]:
                count[name][0] += 1
                
# Assume probability of substitution on N(s) base and -(gap) in aligned data is same with probability of substitution on identified bases
# Of course, indeed, the frequency of mutation is different along each position and each gene in aligned data
# But, despite of this fact, I assume that the probability of mutation is independently with each position
# Then, by binomial distribution, the expected number of mutation on N(s) base and -(gap) is L*p
# -> L : the total length of N(s) bases and -(gap)s, p : the probability of substitution on N(s) base and -(gap)

for k in N_or_gap_count.keys():
    sub_len = len(ref_f_coll)-N_or_gap_count[k][0]
    prob_m = count[k][0]/sub_len
    N_or_gap_count[k][0] *= prob_m

Mutation = {name : [] for name in rep.keys()}

for key in list(Mutation.keys()):
    if key != 'NC_045512|China|2019-12-00':
        mutation_count = count[key][0] + round(N_or_gap_count[key][0])
        Mutation[key].append(mutation_count)
    
del Mutation['NC_045512|China|2019-12-00']

In [37]:
# 2-(2) : Calculate mutation rate per year about entire genome

date_dic = {name : [] for name in Mutation.keys()}

for key in Mutation.keys():
    if key != 'MT326173|USA|2020':
        date = key.split('|')[2]
        days = int(date.split('-')[2])  
        if '2019-12' in date:        
            date_dic[key].append(days)
        elif '2020-01' in date:
            days_01 = days + 31
            date_dic[key].append(days_01)
        elif '2020-02' in date:
            days_02 = days + 62
            date_dic[key].append(days_02)
        elif '2020-03' in date:
            days_03 = days + 91
            date_dic[key].append(days_03)
        elif '2020-04' in date:
            days_04 = days + 122
            date_dic[key].append(days_04)
    else:
        date_dic[key].append(112)
                    
# The date of 'MT326173|USA|2020' assume 2020-3-21 from phylogenetic tree(NJ) and pairwise genetic distance in above code

Mutation_rate = {name : [] for name in Mutation.keys()}

for key in Mutation.keys():
    if key != 'MN908947|China|2019-12-00':
        m_rate = (Mutation[key][0]/date_dic[key][0])*365 # Mutation rate per year with assuming that 'one year=365 days'
        Mutation_rate[key].append(m_rate)

del Mutation_rate['MN908947|China|2019-12-00']

sum_mutation_rate = 0
for name in Mutation_rate.keys():
    sum_mutation_rate += Mutation_rate[name][0]
    
avg_mutation_rate_per_year = sum_mutation_rate/744
print(avg_mutation_rate_per_year)

22.252475722154607


In [38]:
# 3 : Is there genes have higher mutation rate?
# 3-(1) Define the function that calculate mutation rate per year

def cal_m_rate(rep_align):
    rep = read_fasta(rep_align)

    ref_f_coll = rep['NC_045512|China|2019-12-00']

    count = {name : [0] for name in rep.keys()}
    N_or_gap_count = {name : [0] for name in rep.keys()}
    del count['NC_045512|China|2019-12-00']
    del N_or_gap_count['NC_045512|China|2019-12-00']

    for name in list(rep.keys()):
        if name != 'NC_045512|China|2019-12-00':
            for i in range(len(ref_f_coll)):
                if rep[name][i] == 'N' or ref_f_coll[i] == 'N' or rep[name][i] == '-' or ref_f_coll[i] == '-':
                    N_or_gap_count[name][0] += 1
                elif rep[name][i] != ref_f_coll[i]:
                    count[name][0] += 1

    for k in N_or_gap_count.keys():
        sub_len = len(ref_f_coll)-N_or_gap_count[k][0]
        prob_m = count[k][0]/sub_len
        N_or_gap_count[k][0] *= prob_m

    Mutation = {name : [] for name in rep.keys()}

    for key in list(Mutation.keys()):
        if key != 'NC_045512|China|2019-12-00':
            mutation_count = count[key][0] + round(N_or_gap_count[key][0])
            Mutation[key].append(mutation_count)
    
    del Mutation['NC_045512|China|2019-12-00']
    
    date_dic = {name : [] for name in Mutation.keys()}

    for key in Mutation.keys():
        if key != 'MT326173|USA|2020':
            date = key.split('|')[2]
            days = int(date.split('-')[2])  
            if '2019-12' in date:        
                date_dic[key].append(days)
            elif '2020-01' in date:
                days_01 = days + 31
                date_dic[key].append(days_01)
            elif '2020-02' in date:
                days_02 = days + 62
                date_dic[key].append(days_02)
            elif '2020-03' in date:
                days_03 = days + 91
                date_dic[key].append(days_03)
            elif '2020-04' in date:
                days_04 = days + 122
                date_dic[key].append(days_04)
        else:
            date_dic[key].append(112)
                    

    Mutation_rate = {name : [] for name in Mutation.keys()}

    for key in Mutation.keys():
        if key != 'MN908947|China|2019-12-00':
            m_rate = (Mutation[key][0]/date_dic[key][0])*365 
            Mutation_rate[key].append(m_rate)

    del Mutation_rate['MN908947|China|2019-12-00']

    sum_mutation_rate = 0
    for name in Mutation_rate.keys():
        sum_mutation_rate += Mutation_rate[name][0]
    
    avg_mutation_rate_per_year = sum_mutation_rate/744
    return avg_mutation_rate_per_year

In [39]:
# 3-(2) Mutation rate per year about each gene
# > ORF1ab

r_ORF1ab = cal_m_rate('SARS_CoV2_ORF1ab_mul_al.fa')
print(r_ORF1ab)

# > S gene

r_S = cal_m_rate('SARS_CoV2_S_mul_al.fa')
print(r_S)

# > ORF3a

r_ORF3a = cal_m_rate('SARS_CoV2_ORF3a_mul_al.fa')
print(r_ORF3a)

# > E gene

r_E = cal_m_rate('SARS_CoV2_E_mul_al.fa')
print(r_E)

# > M gene

r_M = cal_m_rate('SARS_CoV2_M_mul_al.fa')
print(r_M)

# > ORF6

r_ORF6 = cal_m_rate('SARS_CoV2_ORF6_mul_al.fa')
print(r_ORF6)

# > ORF7a

r_ORF7a = cal_m_rate('SARS_CoV2_ORF7a_mul_al.fa')
print(r_ORF7a)

# > ORF8

r_ORF8 = cal_m_rate('SARS_CoV2_ORF8_mul_al.fa')
print(r_ORF8)

# > N gene

r_N = cal_m_rate('SARS_CoV2_N_mul_al.fa')
print(r_N)

14.908217558855267
2.223296638044254
1.6579057890185043
0.04539623541727601
0.2442921377530046
0.15668024938156033
0.12571141916870077
1.940535160528402
0.9968136308366995


In [40]:
# 3-(3) Can you explain this situation?
# Detailed explanation is written on my midterm homework word file

# Define the function calculating the length of gene
def cal_len(rep_align):
    rep = read_fasta(rep_align)

    ref_f_coll = rep['NC_045512|China|2019-12-00']
    
    L_ref = len(ref_f_coll)
    
    return L_ref
    
# The result of each gene length
L_ORF1ab = cal_len('SARS_CoV2_ORF1ab_mul_al.fa')
print(L_ORF1ab)

L_S = cal_len('SARS_CoV2_S_mul_al.fa')
print(L_S)

L_ORF3a = cal_len('SARS_CoV2_ORF3a_mul_al.fa')
print(L_ORF3a)

L_E = cal_len('SARS_CoV2_E_mul_al.fa')
print(L_E)

L_M = cal_len('SARS_CoV2_M_mul_al.fa')
print(L_M)

L_ORF6 = cal_len('SARS_CoV2_ORF6_mul_al.fa')
print(L_ORF6)

L_ORF7a = cal_len('SARS_CoV2_ORF7a_mul_al.fa')
print(L_ORF7a)

L_ORF8 = cal_len('SARS_CoV2_ORF8_mul_al.fa')
print(L_ORF8)

L_N = cal_len('SARS_CoV2_N_mul_al.fa')
print(L_N)

21349
3822
828
228
669
186
366
366
1260
